# Post-processing of case studies

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

It will look for results in a "simulation/<simulator>" folder on the same level as this notebook. Simulators currently supported: "dymmola", "optimica"

## Import required libraries

In [1]:
import os
import cases

import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from buildingspy.io.outputfile import Reader
from buildingspy.io.postprocess import Plotter

## Load in simulation results

In [2]:
# Helper methods

def get_results(case_name, simulator = "dymola"):
    """ Get the results for the case with name `case_name`
    from a "simulation/<simulator>" folder.

    Parameters:
    
        simulator: str
            "optimica" or "dymola"

    Returns:

        Reader: buildingspy.io.outputpy.Reader instance
    """
    if simulator not in ["dymola", "optimica"]:
        raise Exception("Unsupported simulator provided.")

    # First make sure that the simulation was successful

    # For results generated using Dymola
    if simulator == "dymola":

        dslog_name = os.path.join("simulations", simulator, case_name, "dslog.txt")
        with open(dslog_name) as dslog:
            if not "Integration terminated successfully" in dslog.read():
                raise Exception("Simulation failed. Check {}".format(dslog_name))

    # For results generated using Optimica
    elif simulator == "optimica":

        for file in os.listdir(os.path.join("simulations", simulator, case_name)):
            if '.json' in file:
                json_log_fullpath = os.path.join("simulations", simulator, case_name, file)
                import json
                if not json.load(open(json_log_fullpath))["translation"]["success"]:
                    raise Exception("Simulation failed. Check {}".format(json_log_fullpath))
    
    else:
        raise Exception("Simulation log found for neither supported simulation engine.")

    file_name = cases.get_result_fullpath(case_name, simulator)

    return Reader(file_name, simulator)


def get_partial_results(case_name, list_of_variables, simulator = "dymola", as_dataframe = False):
    """ Get a dictionary with the variable names and the time series for `list_of_variables`

    Parameters:
    
        case_name: str
            Name of the simulated case

        list_of_variables: list
            Variable names for which to extract the result data

        simulator: str
            "optimica" or "dymola" 

        as_dataframe: bool
            If all variables have the same time record in the output one may convert the 
            resulting dictionary into a dataframe

    Returns:

        d: dataframe or dict
            holds time and varable for each variable listed in list_of_variables
    """
    reader = get_results(case_name, simulator = simulator)
    d = dict()
    for v in list_of_variables:
        d['time_' + v] = reader.values(v)[0]
        d[v] = reader.values(v)[1]

    if as_dataframe:
        import pandas as pd
        d = pd.DataFrame(d)

    return d

## Explore variables found in the simulation results

In [108]:
# Read in all simulation results

all_case_names = cases.get_list_of_case_names()
print(all_case_names)

res = {"optimica" : dict(), "dymola" : dict()}

simulators = ["dymola", "optimica"]
for simulator in simulators:
    for case_name in all_case_names:
        res[simulator][case_name] = get_results(case_name, simulator)

print("Read in simulation results.")

['base_test', '1711_cwreset_test', '1711_wse_test', '1711_cwreset_wse_test']
Read in simulation results.


In [102]:
# aggregate data timeseries

PAgg = {"dymola": dict(), "optimica": dict()}
simulators = ["dymola", "optimica"]
for simulator in simulators:
    for case in all_case_names:
        PAgg[simulator][case] = get_partial_results(
            case,  ["PConWatPumAgg.y", "PChiWatPumAgg.y", "PChiAgg.y", "PCooTowAgg.y", "PSupFanAgg.y","QRooIntGaiAgg.y"], simulator, as_dataframe=True)
        PAgg[simulator][case] = PAgg[simulator][case].loc[(PAgg[simulator][case].index % 2)==0,:].reset_index(drop=True)

In [103]:
PAgg["dymola"]['base_test'].head(30)

,time_PConWatPumAgg.y,PConWatPumAgg.y,time_PChiWatPumAgg.y,PChiWatPumAgg.y,time_PChiAgg.y,PChiAgg.y,time_PCooTowAgg.y,PCooTowAgg.y,time_PSupFanAgg.y,PSupFanAgg.y,time_QRooIntGaiAgg.y,QRooIntGaiAgg.y
0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
1,175.888962,0.000000e+00,175.888962,4.672157e+04,175.888962,0.000,175.888962,1.055334e+06,175.888962,7.466347e+06,175.888962,2.148398e+05
2,623.988892,0.000000e+00,623.988892,1.657508e+05,623.988892,0.000,623.988892,3.743933e+06,623.988892,2.648783e+07,623.988892,2.703924e+06
3,1200.000000,1.007340e+07,1200.000000,2.864085e+05,1200.000000,991351.375,1200.000000,7.200000e+06,1200.000000,5.093905e+07,1200.000000,1.000002e+07
4,1200.829224,1.008203e+07,1200.829224,2.865635e+05,1200.829224,991351.375,1200.829224,7.204975e+06,1200.829224,5.097425e+07,1200.829224,1.001384e+07
5,3456.000000,4.404328e+07,3456.000000,7.906597e+05,3456.000000,991351.375,3456.000000,2.073600e+07,3456.000000,1.467045e+08,3456.000000,8.294402e+07
6,6912.000000,9.608814e+07,6912.000000,1.563176e+06,6912.000000,991351.375,6912.000000,4.147200e+07,6912.000000,2.934089e+08,6912.000000,3.317760e+08
7,10368.000000,1.481330e+08,10368.000000,2.335692e+06,10368.000000,991351.375,10368.000000,6.220800e+07,10368.000000,4.401134e+08,10368.000000,7.464960e+08
8,13824.000000,2.001779e+08,13824.000000,3.108209e+06,13824.000000,991351.375,13824.000000,8.294400e+07,13824.000000,5.868179e+08,13824.000000,1.327104e+09
9,17280.000000,2.522227e+08,17280.000000,3.880726e+06,17280.000000,991351.375,17280.000000,1.036800e+08,17280.000000,7.335223e+08,17280.000000,2.073600e+09


In [109]:
pumCW = {"dymola": dict(), "optimica": dict()}
simulators = ["dymola", "optimica"]

for simulator in simulators:
    
    for case in all_case_names:
        if (case == "1711_cwreset_test") or (case == "1711_cwreset_wse_test"):
            var_labels = ["pumCW.m_flow", "pumCW.VMachine_flow", "pumCW.eta", "pumCW.y","pumCW.P","TAirSup.T"]
        else:
            var_labels = ["pumCW.m_flow", "pumCW.VMachine_flow", "pumCW.P", "pumCW.eta","TAirSup.T"]
            
        pumCW[simulator][case] = get_partial_results(
            case,  var_labels, simulator, as_dataframe=True)
        
        pumCW[simulator][case] = pumCW[simulator][case].loc[(pumCW[simulator][case].index % 2)==0,:].reset_index(drop=True)

['base_test', '1711_cwreset_test', '1711_wse_test', '1711_cwreset_wse_test']

In [106]:
pumCW["dymola"]["1711_cwreset_test"].head(20)
pumCW["dymola"]["1711_cwreset_test"].max()

time_pumCW.m_flow           1.728000e+06
pumCW.m_flow                1.568687e+01
time_pumCW.VMachine_flow    1.728000e+06
pumCW.VMachine_flow         1.575642e-02
time_pumCW.eta              1.728000e+06
pumCW.eta                   4.900000e-01
time_pumCW.y                1.728000e+06
pumCW.y                     1.000000e+00
time_pumCW.P                1.728000e+06
pumCW.P                     6.429914e+02
time_TAirSup.T              1.728000e+06
TAirSup.T                   3.001697e+02
dtype: float32

In [107]:
pumCW["dymola"]["base_test"].max()

time_pumCW.m_flow           1.728000e+06
pumCW.m_flow                3.968254e+01
time_pumCW.VMachine_flow    1.728000e+06
pumCW.VMachine_flow         3.985848e-02
time_pumCW.P                1.728000e+06
pumCW.P                     1.748821e+04
time_pumCW.eta              1.728000e+06
pumCW.eta                   4.900000e-01
time_TAirSup.T              1.728000e+06
TAirSup.T                   3.002609e+02
dtype: float32

# Compile total energy use per device, case, and simulator

### Energy use

We obtain the energy use as cumulative power consumed by:

- Zone supply fan: fan.P
- Chilled water pump: pumCHW.P
- Condenser water pump: pumCW.P
- Chiller: chi.P
- Cooling tower: cooTow.PFan

We also look at the aggregate IT Heat Load

In [56]:
vlmap = {
    "PConWatPumAgg.y" : "CW Pump [kWh]",
    "PChiWatPumAgg.y" : "CHW Pump[kWh]",
    "PChiAgg.y" : "Chiller [kWh]",
    "PCooTowAgg.y" : "Cooling Tower Fan [kWh]",
    "PSupFanAgg.y" : "Supply Fan [kWh]",
    "QRooIntGaiAgg.y" : "IT Heat Load [kWh]"
}

# Season: use value from the case dist

# Controller design: use long name of the case dict

# *mg add season later

# PAgg_compiled = pd.DataFrame(columns = [["Simulator", "Controller Design", "Season"] + list(vlmap.values())])
PAgg_compiled = pd.DataFrame()

inx = 0

for simulator in simulators:
    for case in cases.get_cases():
        PAgg_compiled.loc[inx, "Simulator"] = simulator
        PAgg_compiled.loc[inx, "Controller Design"] = case['long_name']
        PAgg_compiled.loc[inx, "Season"] = case['season']
        for var in list(vlmap.keys()):
            # grab the last value in the cumulative result variables and convert to kWh from Ws
            PAgg_compiled.loc[inx, vlmap[var]] = (PAgg[simulator][case['name']].loc[:,var][-1:].values[0]/(1000*3600)).round(1)

        inx += 1
        
PAgg_compiled["Total Energy Use [kWh]"] = PAgg_compiled.loc[:,vlmap['PConWatPumAgg.y'] : vlmap["PSupFanAgg.y"]].sum(axis = 1)

In [57]:
PAgg_compiled

,Simulator,Controller Design,Season,CW Pump [kWh],CHW Pump[kWh],Chiller [kWh],Cooling Tower Fan [kWh],Supply Fan [kWh],IT Heat Load [kWh],Total Energy Use [kWh]
0,dymola,Base-case,test,6647.1,664.1,609.8,2880.0,20375.6,237500.0,31176.6
1,dymola,Alternative CW reset,test,256.4,517.7,0.5,2879.0,20375.6,237500.0,24029.2
2,dymola,Alternative WSE on/off,test,6649.4,664.1,609.7,2880.0,20375.6,237500.0,31178.8
3,dymola,Alternative CW reset and WSE on/off,test,256.6,517.7,0.0,2880.0,20375.6,237500.0,24029.9
4,optimica,Base-case,test,6647.1,664.1,609.8,2880.0,20375.6,237500.0,31176.6
5,optimica,Alternative CW reset,test,256.4,517.7,0.5,2879.0,20375.6,237500.0,24029.2
6,optimica,Alternative WSE on/off,test,6649.4,664.1,609.7,2880.0,20375.6,237500.0,31178.8
7,optimica,Alternative CW reset and WSE on/off,test,256.6,517.7,0.0,2880.0,20375.6,237500.0,24029.9


## Energy performance metrics: PUE

In [58]:
pue_df = PAgg_compiled.copy()
pue_df.drop(labels = "IT Heat Load [kWh]", axis = 1, inplace = True)
pue_df.loc[:,vlmap['PConWatPumAgg.y'] : "Total Energy Use [kWh]"] = PAgg_compiled.loc[:,vlmap['PConWatPumAgg.y'] : "Total Energy Use [kWh]"].divide(
    PAgg_compiled[vlmap["QRooIntGaiAgg.y"]], axis = 'rows')
pue_df["1/PUE"] = 1/pue_df['Total Energy Use [kWh]']
pue_df

,Simulator,Controller Design,Season,CW Pump [kWh],CHW Pump[kWh],Chiller [kWh],Cooling Tower Fan [kWh],Supply Fan [kWh],Total Energy Use [kWh],1/PUE
0,dymola,Base-case,test,0.027988,0.002796,0.002568,0.012126,0.085792,0.131270,7.617893
1,dymola,Alternative CW reset,test,0.001080,0.002180,0.000002,0.012122,0.085792,0.101176,9.883808
2,dymola,Alternative WSE on/off,test,0.027997,0.002796,0.002567,0.012126,0.085792,0.131279,7.617355
3,dymola,Alternative CW reset and WSE on/off,test,0.001080,0.002180,0.000000,0.012126,0.085792,0.101179,9.883520
4,optimica,Base-case,test,0.027988,0.002796,0.002568,0.012126,0.085792,0.131270,7.617893
5,optimica,Alternative CW reset,test,0.001080,0.002180,0.000002,0.012122,0.085792,0.101176,9.883808
6,optimica,Alternative WSE on/off,test,0.027997,0.002796,0.002567,0.012126,0.085792,0.131279,7.617355
7,optimica,Alternative CW reset and WSE on/off,test,0.001080,0.002180,0.000000,0.012126,0.085792,0.101179,9.883520


## Total energy use and PUE comparative plots

In [59]:
df = pd.melt(PAgg_compiled, id_vars=["Simulator", "Controller Design", "Season"], value_vars = vlmap.values(), var_name="Device", value_name='Energy Use [kWh]')

In [61]:
from matplotlib import cm
cmap = cm.get_cmap('Spectral')

plot = PAgg_compiled.loc[:,:vlmap['PSupFanAgg.y']].set_index(['Controller Design', 'Simulator']).plot(
    kind='bar', x = None, stacked=True,
    title = "Power consumption per device, controller design, and simulator, in kWh",
    rot = 45, cmap=cmap).legend(loc='upper right')

fig = plot.get_figure()
fig.set_size_inches(8, 8)
fig.tight_layout()
fig.savefig("img/energy_use.png", bbox_inches='tight')

plot = pue_df.loc[:,:vlmap['PSupFanAgg.y']].set_index(['Controller Design', 'Simulator']).plot(
    kind='bar', x = None, stacked=True,
    title = "PUE per device, controller design, and simulator, in kWh",
    rot = 45, cmap=cmap).legend(loc='upper right')

fig = plot.get_figure()
fig.set_size_inches(8, 8)
fig.tight_layout()
fig.savefig("img/pue.png", bbox_inches='tight')

## Example to access timeseries results for each variable

In [189]:
# Look at some resulting variables
cases.get_list_of_case_names()
# res['optimica']['test_base'].varNames()
(t, P_agg) = res['dymola']['base_test'].values("QRooIntGaiAgg.y")
P_agg[-1]

855000000000.0

## Configure timeseries plots

In [202]:
plt.rcParams['axes.facecolor']='whitesmoke'
plt.rcParams['font.size'] = 6
plt.rcParams['text.usetex'] = False
plt.rcParams['legend.facecolor'] = 'white'
plt.rcParams['legend.framealpha'] = 0.75
plt.rcParams['legend.edgecolor'] = 'none'
plt.rcParams['savefig.dpi'] = 300

def save_plot(figure, file_name):
    """ Save the figure to a pdf and png file in the directory `img`
    """
    import os
    import matplotlib.pyplot as plt
    
    out_dir = "img"
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    figure.savefig(os.path.join(out_dir, '{}.pdf'.format(file_name)))
    figure.savefig(os.path.join(out_dir, '{}.png'.format(file_name)))
    plt.clf()
    

def configure_axes(axes):
    """ Configure the axis style
    """
    axes.spines['right'].set_visible(False)
    axes.spines['top'].set_visible(False)
    axes.spines['left'].set_visible(False)
    axes.spines['bottom'].set_visible(False)
    axes.grid(color='lightgrey', linewidth=0.25)
    return


# ---------------------------------------------------------------------------
# helper functions and scripts

def set_cases_and_initiate_plot():
    from matplotlib.gridspec import GridSpec
    cases = ['test_base', 'test_1711']
    seasons = ['']
    num_cases = len(cases)
    num_seasons = len(seasons)
    
    fig = plt.figure(figsize=(6.5,8.))
    gs1 = GridSpec(80, 1)
    gs1.update(left=0.1, right=0.9, hspace=0.05)
    
    ax = list()
    ax.insert(0, fig.add_subplot(gs1[0:11,:]))
    ax.insert(1, fig.add_subplot(gs1[12:23,:]))
    ax.insert(2, fig.add_subplot(gs1[28:39,:]))
    ax.insert(3, fig.add_subplot(gs1[40:51,:]))
    ax.insert(4, fig.add_subplot(gs1[56:67,:]))
    ax.insert(5, fig.add_subplot(gs1[68:79,:]))
    
    # fig, ax = plt.subplots(nrows=num_cases*num_seasons, ncols=1, figsize = (6.5,8.))
    # fig, ax = plt.subplots(nrows=20, ncols=1, figsize = (6.5,8.))
    
    return cases, seasons, num_cases, num_seasons, fig, ax

def set_title(ax, title):
    left, width = .01, .97
    bottom, height = .01, .88
    right = left + width
    top = bottom + height
    
    title_str = r"$\it{" + title + "}$"
    ax.text(left, top,
            title_str,
            verticalalignment = 'center',
            horizontalalignment = 'left', 
            transform=ax.transAxes,
            fontsize = 6, color = 'k',
            bbox=dict(facecolor='white', alpha=0.75, edgecolor='none'))
    
    
def set_up_labels(i, ax, cases, seasons, num_cases, num_seasons, x_axis_label, y_axis_label):
    # Hide xtick labels and ticks on the upper case subplot (each basecase)
    if i % 2 == 0:
        hide_tick_labels(ax)

    # Print x axis title only below the lowest subplot
    if i  == num_cases*num_seasons - 1:
        ax.set_xlabel(x_axis_label)
    ax.set_ylabel(y_axis_label)
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))
     
    # Annotate case
    set_title(ax, cases[i % 2])
    # Annotate case
    # if i % 2 == 0:
    #     title_str = r"$\bf{" + seasons[i/2] + "}$" + ' (upper: ' + r"$\it{" + cases[i % 2] + "}$" + ', lower: ' + r"$\it{"  + cases[(i-1) % 2] + "}$" + ')'
    #     ax.set_title(title_str, # mg assign appropriate season/case
    #                  verticalalignment = 'top',
    #                  horizontalalignment = 'center', 
    #                  fontsize = 6, color = 'k')
        
    # Print legend only at the lower plot (g36 case)
    if i % 1 == 0:
        ax.legend(loc='center right', ncol=1)
    configure_axes(ax)
        
    #plt.tight_layout(h_pad=0)
    plt.tight_layout()
    #plt.subplots_adjust(hspace = .2)
        
def tem_conv_CtoF(T_in_degC):
    '''Converts temperature provided in degC to degF
    '''
    T_in_degF = (T_in_degC)*9./5. + 32.
    
    return T_in_degF
        
def add_secondary_yaxis_for_degF(ax, time, temp_in_K):
        # Add a secondary axis with temperatures represented in F
        ax_F = ax.twinx()
        # Get limits to match with the left axis
        ax_F.set_ylim([tem_conv_CtoF(ax.get_ylim()[0]),tem_conv_CtoF(ax.get_ylim()[1])])
        # plot a "scaler" variable and make it invisible
        ax_F.plot(time, tem_conv_CtoF(temp_in_K-273.15), linewidth=0.0)
        ax_F.set_ylabel('temperature [$^\circ$F]')
        configure_axes(ax_F)
        #ax.grid(False)
        #ax.xaxis.grid()
        
def hide_tick_labels(ax):
    '''Removes labels and ticks. Kwargs: bottom controls the ticks, labelbottom the tick labels
    '''
    ax.tick_params(axis = 'x',labelbottom='off',bottom='off')

## Plot results

## Chiller plant - power

*mg adjust this method to take various cases as needed

In [203]:
def plot_power(res_set):
    ''' Main method that plots the results
    '''
    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 6}
    matplotlib.rc('font', **font)
    
    plt.clf()
    
    time_scale=3600.
       
    (t, pumCHW_P) = res_set.values("pumCHW.P")
    (t, pumCW_P) = res_set.values("pumCW.P")
    (t, chi_P) = res_set.values("chi.P")
    t = t/time_scale
   
    # Plot figure
    fig = plt.figure(figsize=(6.5, 2.5))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, pumCHW_P, label = '$P_{chw,p}$', linewidth=0.5)
         
    # make_ticklabels_invisible(plt.gcf())
        
    # customize days to display

    ax.set_xlabel('time [h]')
    ax.set_ylabel('Chiller water pump power [W]')
    #ax.xaxis.set_ticks(np.arange(min(t)+0, 365, 1))

    ax.set_xlim([min(t), min(t)+24])
    ax.set_xticks(range(24))
 
    ax.legend(loc='center right', ncol=1)

    configure_axes(ax)
        
    return plt

# Create the plot for all seasons and cases
fig = plot_power(res["dymola"]["base_test"])
save_plot(fig, "CHWpum_P")

### Outside conditions

In [208]:
def plot_outside(res_set):
    # ------------------------------------------------------
    plt.clf()
    plt.ylim([0., 20.])
    time_scale=86400.
    # ------------------------------------------------------
    
    ax = list()
    # get a list of unique climate cases
    unique_climate_cases = list(res_set.keys())[0] # all in the same climate
    
    subplot_id = 0
    for climate in unique_climate_cases:
        (t, TOut) = res_set[case_name].values("weaBus.TDryBul")
        (t, TOutWet) = res_set[case_name].values("weaBus.TWetBul")
        
        t = t/time_scale
        
        # Generate figure and plot data
        ax.insert(subplot_id, plt.subplot(len(unique_climate_cases)*2, 1, subplot_id+1))
        
        for subplot_id in [0,2,4]:
            ax[subplot_id].plot(t, TOut-273.15, 'r', \
                   label='$T_{oa}$', linewidth=0.5)
            ax[subplot_id].set_ylabel('temperature [$^\circ$C]')
            add_secondary_yaxis_for_degF(ax[subplot_id], t, TOut)
            set_title(ax[subplot_id], '$T_{oa}$')
        
        subplot_id += 1
        
        for subplot_id in [1,3,5]:
            ax[subplot_id].plot(t, TOutWet-273.15, 'r', \
                   label='$T_{oa,w}$', linewidth=0.5)
            ax[subplot_id].set_ylabel('temperature [$^\circ$C]')
            add_secondary_yaxis_for_degF(ax[subplot_id], t, TOutWet)
            set_title(ax[subplot_id], '$T_{oa,w}$')
        
        # customize days to display
        ax[subplot_id].set_xlim([min(t)+5, min(t)+10])

        configure_axes(ax[subplot_id])
        ax[subplot_id].legend(loc='center right', ncol=1)
        
        subplot_id += 1

#     ax[5].set_xlabel('time [days]')

    return plt

# Create the plot for all seasons and cases
fig = plot_outside(res["dymola"]["base_test"])
save_plot(fig, "outside")

AttributeError: 'Reader' object has no attribute 'keys'

cases.get_list_of_case_names()


In [209]:
res[case].values("fan.P")

TypeError: unhashable type: 'dict'

## Plot energy use with diverse loads

In [ ]:
def plot_all_energy(r_base_without, r_g36_without, r_base_with, r_g36_with):
    from buildingspy.io.outputfile import Reader
    import matplotlib.pyplot as plt
    
    plt.clf()    

    # Conversion to kWh/m2
    conv = 1/3600./1000.
    results = [r_base_without, r_g36_without, r_base_with, r_g36_with]
    width = 0.5       # the width of the bars: can also be len(x) sequence

    hea    = [0., 0., 0., 0.]
    cooSen = [0., 0., 0., 0.]
    cooLat = [0., 0., 0., 0.]
    fan    = [0., 0., 0., 0.]
    cooLatBas = [0., 0., 0., 0.]
    coo = [0, 0, 0., 0.]
    fanBas = [0., 0., 0., 0.]
    
    idx = [0, 1, 2, 3]
    for i in idx:
        res_dic = results[i]
        hea[i]    =  res_dic['res.EHea'][-1] * conv / COPh
        cooSen[i] = -res_dic['res.ECooSen'][-1] * conv / COPc
        cooLat[i] = -res_dic['res.ECooLat'][-1] * conv / COPc
        coo[i] = cooSen[i] + cooLat[i]
        fan[i]    =  res_dic['res.EFan'][-1] * conv
        cooLatBas[i] = hea[i] + cooSen[i]
        fanBas[i]    = cooLatBas[i] + cooLat[i]
    
    
    plt.figure(figsize=(5,3))

    p1 = plt.bar(idx, hea,    width, color='r')
    p2 = plt.bar(idx, cooSen, width, bottom=hea, color='g')
    p3 = plt.bar(idx, cooLat, width, bottom=cooLatBas, color='b')
    p4 = plt.bar(idx, fan,    width, bottom=fanBas, color='k')
    plt.ylabel('site electricity use [kWh/(m2  a)]')
    plt.xticks([0, 1, 2, 3], ('base case', 'guideline 36', 'base case, $\pm 50\%$', 'guideline 36, $\pm 50\%$'))
    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0], p3[0], p4[0]), \
               ('heating', 'sensible cooling', 'latent cooling', 'fan'), \
              loc='upper right')
    plt.grid(color='lightgrey', axis='y', linewidth=0.25)
    for spine in plt.gca().spines.values():
        spine.set_visible(False)
    save_plot(plt, "energy_all")
    
plot_all_energy(r_base_annual_without_diverse_loads, r_g36_annual_without_diverse_loads, \
                r_base_annual_with_diverse_loads,    r_g36_annual_with_diverse_loads)

### Diagnostic output

In [ ]:
print("CPUtime, base {} h".format(r_base_annual_without_diverse_loads['CPUtime'][-1]/3600.))
print("CPUtime, G36  {} h".format(r_g36_annual_without_diverse_loads['CPUtime'][-1]/3600.))

In [ ]:
plt.clf()

roo_lis = [{"var": 'nor', "name": 'north'},
           {"var": 'wes', "name": 'west'},
           {"var": 'sou', "name": 'south'},
           {"var": 'eas', "name": 'east'},
           {"var": 'cor', "name": 'corridor'}]
nRoo = len(roo_lis)
iRoo = 1
iPlt = 1
for roo in roo_lis:
    iSim = 1
    for sim in [{"res": r_base_annual_without_diverse_loads, 'name': 'base case'}, 
                {"res": r_g36_annual_without_diverse_loads, 'name': 'guideline36'}]:
        plt.subplot(nRoo, 2, iPlt)
        plt=Plotter.boxplot(t=sim['res']['time'], \
                    y=sim['res']['flo.{}.air.vol.T'.format(roo['var'])]-273.15, \
                    increment=3600, nIncrement=24)
        #plt.set_facecolor('mistyrose')    
        #plt.patch.fill_between(t, TSetHea-273.15, y2=TSetCoo-273.15, color='white')
        # Decorate, save and show the plot
        if iRoo is nRoo:
            plt.xlabel('Time [h]') 
        if iSim is 1:
            plt.ylabel(u'$T_{room}$ [$^\circ$C]')
        plt.grid()
        plt.xlim(6.5,19.5)
        plt.ylim(18, 26)
        plt.yticks([18, 20, 22, 24, 26])

        plt.title('{} zone, {}'.format(roo['name'], sim['name']))
        iSim+=1
        iPlt+=1
    iRoo+=1
plt.tight_layout()
save_plot(plt, "roomTemperatures_boxplot")
plt.show()

In [ ]:
a = cases.get_cases()

In [ ]:
a